# Trabalhos concluídos

1. Revisão bibliográfica e texto base
2. Formulação matemática CTP
3. Formulação matemática TI
4. Gerador de instâncias sintéticas
5. Modelo CPLEX para formulação CTP
6. Modelo CPLEX para formulação TI

# Metaheurísticas

## Representação de uma solução viável (ideia inicial)

- Cada sonda guarda uma lista (array) de projetos executados por ela (map <sonda, lista>)

    - guardar projeto e tempos de início e término

- Manter um contador de gastos (geral e por tipo de projeto)

- Cada sonda guarda uma lista de janelas de tempo disponíveis

- Cada sonda guarda uma lista de possíveis projetos (heap, com key por critério guloso)

## Representação de uma solução viável (segunda versão)

- Cada sonda guarda uma lista duplamente encadeada de atividades

    - Atividades podem ser janelas de tempo ou alocações de projetos
    
    - Cada item da lista contém (atividade, inicio, final)

- Manter um contador de gastos (geral e por tipo de projeto)

- Cada sonda guarda uma lista de possíveis projetos (heap, com key por critério guloso)

## Estruturas de dados

- Doubly Linked list: representação da solução

- Heap: candidatos a serem inseridos na solução

# Critérios gulosos

- lucro

- lucro / custo

- lucro / duração

- lucro / distância

## Heurística de construção

1. Escolher uma sonda
2. Escolher um projeto (dentro da heap de possíveis projetos)
3. Alocar na sonda
    - se for inviável (pelo tempo), tentar realocar até k projetos (menos restritivos dentro da janela do projeto a ser inserido)
    - ou tentar realocar por sliding window
4. Atualizar dados (gastos, janelas de tempo, projetos possíveis)
5. Repetir até restrições serem alcançadas


## Heurística de refinamento (busca local)

### Estrutura de vizinhança 1: Inserir projeto ainda não escolhido

- Escolher projeto por critério guloso (dentro dos não selecionados)

- Tentar inserir sem remover ninguém

- Se for inviável, então, dentro da janela do projeto a ser inserido:

    - percorrer todos os projetos e escolher o maior ganho / menor perda, ao inserir o novo projeto (sliding window)
    
    - ou tentar realocar até k projetos (os k menos restritivos dentro da janela do projeto a ser inserido)

### Estrutura de vizinhança 2: Remover k projetos e preencher com heurística de construção

- Posso variar k projetos a serem removidos

- Posso variar critério de remoção e reconstrução para maior diversificação


## Crossover

### Ideia 1: quando a instância tem mais de uma sonda

Dadas duas soluções possíveis:

- Dividir as sondas em dois conjuntos

- Para a primeira solução, escolher o primeiro conjunto de sondas (solução parcial)

- Para a segunda solução, escolher o segundo conjunto de sondas (solução complementar)

- Remover aleatoriamente projetos que estejam tanto na solução parcial, quanto na complementar

- Unir solução parcial com solução complementar

- Para cada sonda

    - Preencher buracos com heurística de construção

### Idéia 2: mais genérica

Dadas duas soluções possíveis, para cada sonda:

- Dividir os projetos em dois grupos

- Para a primeira solução, escolher o primeiro conjunto de projetos (solução parcial)

- Para a segunda solução, escolher o segundo conjunto de projetos (solução complementar)

- Remover aleatoriamente projetos que estejam tanto na solução parcial, quanto na complementar

- Para cada projeto da solução complementar:

    - Tentar inserir na solução parcial usando heurística de construção (projetos já selecionados)
    
- Preencher buracos com heurística de construção

## Decodificador BRKGA

### Representação:

- Cada projeto recebe um número aleatório entre 0 e 1

### Ideia:

- Usar a heurística de construção, simplificada:

    - Critério guloso: ordem dos números aleatórios
    
    - Eficiência: não pegar pesado em tentar viabilizar a inserção de projetos inviáveis

- Construir parte da população inicial com GRASP

## Lendo dados

In [1]:
import copy

In [2]:
ls

 instancia_100projetos_10sondas.dat   instancia_300projetos_20sondas.dat
 instancia_100projetos_20sondas.dat   instancia_300projetos_2sondas.dat
 instancia_100projetos_2sondas.dat    instancia_300projetos_5sondas.dat
 instancia_100projetos_5sondas.dat    instancia_30projetos_2sondas.dat
 instancia_10projetos_2sondas.dat     instancia_500projetos_10sondas.dat
 instancia_200projetos_10sondas.dat   instancia_500projetos_20sondas.dat
 instancia_200projetos_20sondas.dat   instancia_500projetos_2sondas.dat
 instancia_200projetos_2sondas.dat    instancia_500projetos_5sondas.dat
 instancia_200projetos_5sondas.dat    instancia_50projetos_2sondas.dat
 instancia_20projetos_2sondas.dat    'Metaheuristicas Mestrado.ipynb'
 instancia_300projetos_10sondas.dat   synthetic_instance_generator.ipynb


In [3]:
coords_x = []
coords_y = []
bacias = []
nomes = []
maturidades = []
qualidades = []
plays = []
soterramentos = []
pcgnas = []
geracao = []
migracao = []
reservatorio = []
geometria = []
retencao = []
pshc = []
mc_vol = []#print ('sonda: ', sonda, ' inserindo projeto ', projeto, ' nos tempos ', release, ' e ', 
                
mi_vol = []
mc_vpl = []
mi_vpl = []
custos = []
tempos_exec = []
inicio_janela = []
final_janela = []
sondas_x = []
sondas_y = []

data = {
    1: coords_x,
    2: coords_y,
    3: bacias,
    4: nomes,
    5: maturidades,
    6: qualidades,
    7: plays,
    8: soterramentos,
    9: pcgnas,
    10: geracao,
    11: migracao,
    12: reservatorio,
    13: geometria,
    14: retencao,
    15: pshc,
    16: mc_vol,
    17: mi_vol,
    18: mc_vpl,
    19: mi_vpl,
    20: custos,
    21: tempos_exec,
    22: inicio_janela,
    23: final_janela,
    25: sondas_x,
    26: sondas_y
}

with open('instancia_10projetos_2sondas.dat', 'r') as f:
    for i, line in enumerate(f):
        if i == 0:
            (n_projetos, custo_total) = int(line.split()[0]), float(line.split()[1])
        elif i == 24:
            n_sondas = int(line.split()[0])
        else:
            for elem in line.split('[')[1].split(']')[0].split(','):
                try:
                    data[i].append(float(elem))
                except:
                    data[i].append(elem)

In [4]:
n_projetos

10

In [5]:
custo_total

524.1181725752178

In [6]:
n_sondas

2

## Implementação da primeira heurística de construção

In [7]:
def ConstruirSolucao(dados, n_projetos, n_sondas, custo_total, t_init, t_final, delta_t, criterio='lucro/custo'):
    
    import math
    import copy
    import numpy as np
    import heapq
    
    dados_local = copy.deepcopy(dados)
    
    n_periodos = ((t_final - t_init) // delta_t) + 1
    
    # convertendo dados de tempo para períodos -------------- TODO: verificar se não está somando 1 a mais
    # ------------------------------------------------------- TODO: posso fazer isso como pré-processamento
    for i in range(n_projetos):
        dados_local[21][i] = (dados_local[21][i] // delta_t) + 1
        dados_local[22][i] = (dados_local[22][i] // delta_t)
        dados_local[23][i] = (dados_local[23][i] // delta_t)
    
    # calculando tempo de deslocamento entre projetos: ---------- TODO: posso fazer isso como pré-processamento
    desloc = []
    lag = n_sondas
    for i in range(n_projetos+n_sondas):
        desloc.append([])
        for j in range(n_projetos+n_sondas):
            if (i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[25][j])**2 + (dados[26][i] - dados[26][j])**2 )
            elif (i<n_sondas) and not(j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[1][j-lag])**2 + (dados[25][i] - dados[2][j-lag])**2 )
            elif not(i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[1][i-lag] - dados[26][j])**2 + (dados[2][i-lag] - dados[26][j])**2 )
            else:
                dist = math.sqrt( (dados[1][i-lag] - dados[1][j-lag])**2 + (dados[2][i-lag] - dados[2][j-lag])**2 )
            
            # converte para períodos: --------------------- TODO : verificar se não está somando 1 a mais
            if delta_t == 1:
                desloc[i].append(dist)
            else:
                desloc[i].append((dist // delta_t) + 1)
    
    # inicializando solução
    s = {i:[] for i in range(n_sondas)}
    
    # inicializando janelas de tempo, por sonda
    s_janelas = {i:[[-1, [0, n_periodos]]] for i in range(n_sondas)}
    
    # inicializando lista de candidatos, por sonda
    s_candidatos = {i:[] for i in range(n_sondas)}
    
    # armazena lista de candidatos como uma heap: ------- OBS: valores keys multiplicados por -1 (MaxHeap)
    # ---------------------------------- TODO: posso ordenar projetos pelo critério guloso como pré-processamento?
    for i in range(n_sondas):
        for j in range(n_projetos):
            criterio_val = dados_local[19][j] / dados_local[20][j]
            heapq.heappush( s_candidatos[i], (-criterio_val, j) )
    
    # inicializa contador de gastos total
    gastos = 0
    
    proj_usados = set()
    sondas = set([i for i in range(n_sondas)])
    
    fitness = 0
    
    while (gastos < custo_total and sondas):
        
        # escolher uma sonda
        sonda = np.random.choice(list(sondas))
        
        # escolher um projeto
        projeto = heapq.heappop(s_candidatos[sonda])[1]
        
        # se sonda não tem mais candidatos, remover ela da lista
        if s_candidatos[sonda] == []:
            sondas.remove(sonda)
        
        # se projeto já foi escolhido, pular
        if (projeto in proj_usados):
            continue
        
        # se existe janela disponível na sonda
        aloc = False
        proc = dados_local[21][projeto]
        for i, (prev, janela) in enumerate(s_janelas[sonda]):
            release = max(janela[0], dados_local[22][projeto])
            due = min(janela[1], dados_local[23][projeto])
            if prev == -1:
                setup = desloc[sonda][projeto + lag]
            else:
                setup = desloc[prev + lag][projeto + lag]
            if ( release + proc + setup <= due ):
                
                #print ('sonda: ', sonda, ' inserindo projeto ', projeto, ' nos tempos ', release, ' e ', 
                #       release+setup+proc, ' dentro da janela ', janela[0], janela[1])
                
                # alocar
                s[sonda].append((copy.copy(projeto), copy.copy(release), copy.copy(release+setup+proc)))
                proj_usados.add(copy.copy(projeto))
                fitness += dados_local[19][projeto]
                aloc = True
                
                # atualizar janela da sonda
                
                if ( (int(janela[0])==int(release)) and (int(janela[1])==int(release+setup+proc)) ):
                    # remover janela
                    s_janelas[sonda].pop(i)
                    
                    # print ('janela toda preenchida pelo projeto')
                    
                elif (int(janela[0])==int(release)):
                    # atualizar release da janela e colocar o prev como o projeto alocado
                    s_janelas[sonda][i][0] = copy.copy(projeto)
                    s_janelas[sonda][i][1][0] = copy.copy(release + setup + proc)
                    
                    #print ('projeto preenche inicio da janela')
                    
                elif (int(janela[1])==int(release+setup+proc)):
                    # atualiza due da janela, mantendo o prev
                    s_janelas[sonda][i][1][1] = copy.copy(release)
                    
                    #print ('projeto preenche final da janela')
                    
                else:
                    # abrir janela antes da release, cujo prev é o prev original
                    s_janelas[sonda].insert(i, [copy.copy(prev), [copy.copy(janela[0]), copy.copy(release)]])
                    # e atualizar prev e release da janela original
                    s_janelas[sonda][i+1][0] = copy.copy(projeto)
                    s_janelas[sonda][i+1][1][0] = copy.copy(release + setup + proc)
                    
                    #print ('projeto preenche meio da janela')
                    
                break
            else:
                # senão, tentar realocar projetos ---------------------------------- TODO
                pass
        
        # atualizar dados de gastos
        if (aloc):
            gastos += dados_local[20][projeto]
    
    return fitness, s, s_janelas, s_candidatos

In [8]:
t_init, t_final, delta_t = 0, 5*12*4*7, 7

In [9]:
ConstruirSolucao(data, n_projetos, n_sondas, custo_total, t_init, t_final, delta_t)

(2306.9334613304236,
 {0: [(5, 116.0, 141.0), (6, 15.0, 50.0), (4, 176.0, 214.0), (7, 50.0, 89.0)],
  1: [(9, 156.0, 197.0)]},
 {0: [[-1, [0, 15.0]],
   [7, [89.0, 116.0]],
   [5, [141.0, 176.0]],
   [4, [214.0, 241]]],
  1: [[-1, [0, 156.0]], [9, [197.0, 241]]]},
 {0: [(-1.568047372229324, 1),
   (1.0954842694646125, 8),
   (4.898278979284376, 0),
   (4.058794717521798, 2)],
  1: [(-2.139619506696249, 7),
   (-1.659527893552874, 3),
   (-1.568047372229324, 1),
   (1.0954842694646125, 8),
   (4.898278979284376, 0),
   (4.058794717521798, 2)]})

## Classe para lista duplamente encadeada

In [10]:
class Node:
    
    """
    Documentação:
    """
    
    def __init__(self, data=None):
        self.data = data
        self.next = None
        self.prev = None
    

class DoublyLinkedList:
    
    """
    Documentação:
    """
    
    def __init__(self, nodes=None):
        self.head = None
        if nodes is not None:
            node = Node(data=nodes.pop(0))
            self.head = node
            for elem in nodes:
                new_node = Node(data=elem)
                node.next = new_node
                new_node.prev = node
                node = node.next
    
    def __iter__(self):
        node = self.head
        while node is not None:
            yield node
            node = node.next
    
    def add_node_begining(self, value_to_add):
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        new_node.next = self.head
        self.head.prev = new_node
        self.head = new_node
    
    def add_node_end(self, value_to_add):
        
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        for curr in self:
            pass
        curr.next = new_node
        new_node.prev = curr
    
    def add_node_after_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        new_node.next = node_ref.next
        new_node.prev = node_ref
        node_ref.next.prev = new_node
        node_ref.next = new_node
    
    def add_node_before_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        if node_ref == self.head:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        else:
            node_ref.prev.next = new_node
            new_node.prev = node_ref.prev
            new_node.next = node_ref
            node_ref.prev = new_node
    
    def remove_node_by_ref(self, node):
        if node == self.head:
            node.next.prev = None
            self.head = node.next
            del node
        else:
            node.next.prev = node.prev
            node.prev.next = node.next
            del node
    
    def find_value(self, value):
        for node in self:
            if node.data == value: break
        return node

## Testando lista encadeada

In [11]:
dll = DoublyLinkedList()

In [12]:
dll = DoublyLinkedList(nodes=[1, 2, 3])

In [13]:
for item in dll:
    print (item.data)

1
2
3


In [14]:
dll = DoublyLinkedList()

In [15]:
for item in dll:
    print (item.data)

In [16]:
dll.add_node_begining(value_to_add=1)

In [17]:
for item in dll:
    print (item.data)

1


In [18]:
dll.add_node_begining(value_to_add=2)

In [19]:
for item in dll:
    print (item.data)

2
1


In [20]:
dll.add_node_begining(value_to_add=3)

In [21]:
for item in dll:
    print (item.data)

3
2
1


In [22]:
dll.add_node_begining(value_to_add=4)

In [23]:
for item in dll:
    print (item.data)

4
3
2
1


In [24]:
dll.add_node_end(value_to_add=5)

In [25]:
for item in dll:
    print (item.data)

4
3
2
1
5


In [26]:
dll.add_node_end(value_to_add=6)

In [27]:
for item in dll:
    print (item.data)

4
3
2
1
5
6


In [28]:
node = dll.find_value(1)

In [29]:
node.data

1

In [30]:
node.next.data

5

In [31]:
node.prev.data

2

In [32]:
dll.remove_node_by_ref(node)

In [33]:
for item in dll:
    print (item.data)

4
3
2
5
6


In [34]:
node_ref = dll.find_value(2)

In [35]:
dll.add_node_after_node(value_to_add='X', node_ref=node_ref)

In [36]:
for item in dll:
    print (item.data)

4
3
2
X
5
6


In [37]:
dll.add_node_before_node(value_to_add="Y", node_ref=node_ref)

In [38]:
for item in dll:
    print (item.data)

4
3
Y
2
X
5
6


## Implementação da segunda ideia para heurística de construção

In [42]:
def ConstruirSolucao(dados, n_projetos, n_sondas, custo_total, t_init, t_final, delta_t, criterio='lucro/custo'):
    
    import math
    import copy
    import numpy as np
    import heapq
    
    dados_local = copy.deepcopy(dados)
    
    n_periodos = ((t_final - t_init) // delta_t) + 1
    
    # convertendo dados de tempo para períodos -------------- TODO: verificar se não está somando 1 a mais
    # ------------------------------------------------------- TODO: posso fazer isso como pré-processamento
    for i in range(n_projetos):
        dados_local[21][i] = (dados_local[21][i] // delta_t) + 1
        dados_local[22][i] = (dados_local[22][i] // delta_t)
        dados_local[23][i] = (dados_local[23][i] // delta_t)
    
    # calculando tempo de deslocamento entre projetos: ---------- TODO: posso fazer isso como pré-processamento
    desloc = []
    lag = n_sondas
    for i in range(n_projetos+n_sondas):
        desloc.append([])
        for j in range(n_projetos+n_sondas):
            if (i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[25][j])**2 + (dados[26][i] - dados[26][j])**2 )
            elif (i<n_sondas) and not(j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[1][j-lag])**2 + (dados[25][i] - dados[2][j-lag])**2 )
            elif not(i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[1][i-lag] - dados[26][j])**2 + (dados[2][i-lag] - dados[26][j])**2 )
            else:
                dist = math.sqrt( (dados[1][i-lag] - dados[1][j-lag])**2 + (dados[2][i-lag] - dados[2][j-lag])**2 )
            
            # converte para períodos: --------------------- TODO : verificar se não está somando 1 a mais
            if delta_t == 1:
                desloc[i].append(dist)
            else:
                desloc[i].append((dist // delta_t) + 1)
    
    # inicializando solução
    s = {i:DoublyLinkedList(nodes=[[-1, 0, n_periodos]]) for i in range(n_sondas)}
    
    # inicializando lista de candidatos, por sonda
    s_candidatos = {i:[] for i in range(n_sondas)}
    
    # armazena lista de candidatos como uma heap: ------- OBS: valores keys multiplicados por -1 (MaxHeap)
    # ---------------------------------- TODO: posso ordenar projetos pelo critério guloso como pré-processamento?
    for i in range(n_sondas):
        for j in range(n_projetos):
            criterio_val = dados_local[19][j] / dados_local[20][j]
            heapq.heappush( s_candidatos[i], (-criterio_val, j) )
    
    # inicializa contador de gastos total
    gastos = 0
    
    proj_usados = set()
    sondas = set([i for i in range(n_sondas)])
    
    fitness = 0
    
    while (gastos < custo_total and sondas):
        
        # escolher uma sonda
        sonda = np.random.choice(list(sondas))
        
        # escolher um projeto
        projeto = heapq.heappop(s_candidatos[sonda])[1]
        
        # se sonda não tem mais candidatos, remover ela da lista
        if s_candidatos[sonda] == []:
            sondas.remove(sonda)
        
        # se projeto já foi escolhido, pular
        if (projeto in proj_usados):
            continue
        
        aloc = False
        proc = dados_local[21][projeto]
        
        # percorre atividades da sonda para ver se existe janela disponível na sonda
        for i, node in enumerate(s[sonda]):
            
            # se node não for de janela de tempo, pular
            if (node.data[0] != -1):
                continue
            
            release = max(node.data[1], dados_local[22][projeto])
            due = min(node.data[2], dados_local[23][projeto])
            
            # se node for head, considerar origem da sonda
            if node.prev == None:
                setup = desloc[sonda][projeto + lag]
            # senão, considerar projeto anterior
            else:
                setup = desloc[node.prev.data[0] + lag][projeto + lag]
            
            # se consigo colocar o projeto dentro da janela
            if (release + proc + setup <= due):
                
                #print ('sonda: ', sonda, ' inserindo projeto ', projeto, ' nos tempos ', release, ' e ', 
                #       release+setup+proc, ' dentro da janela ', janela[0], janela[1])
                
                # alocar
                proj_usados.add(copy.copy(projeto))
                fitness += dados_local[19][projeto]
                aloc = True
                inicio = copy.copy(release)
                final = copy.copy(release + setup + proc)
                
                # se projeto preenche janela toda
                if ( (int(node.data[1])==int(inicio)) and (int(node.data[2])==int(final)) ):
                    
                    # substituir janela pela alocação do projeto
                    node.data = [copy.copy(projeto), inicio, final]
                
                # senão, se projeto preenche início da janela    
                elif (int(node.data[1])==int(inicio)):
                    
                    # atualizar release da janela
                    node.data[1] = final
                    
                    # inserir projeto antes da janela
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche final da janela
                elif (int(node.data[2])==int(final)):
                    
                    # atualiza a due da janela
                    node.data[2] = inicio
                    
                    # inserir projeto depois da janela
                    s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche o meio da janela
                else:
                    
                    # atualiza release da janela
                    temp = copy.copy(node.data[1])
                    node.data[1] = final
                    
                    # insere janela nova antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[-1, temp, inicio], node_ref=node)
                    
                    # insere projeto antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                    
                break
            else:
                # senão, tentar realocar projetos ---------------------------------- TODO
                
                # 1. verificar se janela da sonda tem interceção com janela do projeto
                # 2. se sim, 
                #              consigo antecipar o prev? quanto?
                #              consigo postergar o next? quanto?
                #              somando, é viável inserir projeto?
                #              se sim, fazer modificações
                
                
                pass
        
        # atualizar dados de gastos
        if (aloc):
            gastos += dados_local[20][projeto]
    
    return fitness, s, s_candidatos

In [54]:
t_init, t_final, delta_t = 0, 5*12*4*7, 7

In [76]:
f, s, _ = ConstruirSolucao(data, n_projetos, n_sondas, custo_total, t_init, t_final, delta_t)

In [77]:
f

2306.933461330424

In [78]:
for i in range(n_sondas):
    for node in s[i]:
        print (i, node.data)

0 [-1, 0, 15.0]
0 [6, 15.0, 50.0]
0 [-1, 50.0, 176.0]
0 [4, 176.0, 212.0]
0 [-1, 212.0, 241]
1 [-1, 0, 5.0]
1 [7, 5.0, 49.0]
1 [-1, 49.0, 116.0]
1 [5, 116.0, 141.0]
1 [-1, 141.0, 156.0]
1 [9, 156.0, 199.0]
1 [-1, 199.0, 241]


In [ ]:
def BuscaLocalSoft():
    
    return

In [ ]:
def BuscarLocalHard():
    
    return

In [ ]:
def GRASP():
    
    return